In [ ]:
#download GATK tool
wget https://github.com/broadinstitute/gatk/releases/download/4.1.2.0/gatk-4.1.2.0.zip  
#unzip the file and set path
unzip gatk-4.1.2.0.zip
G2
export PATH=$PATH:/data/gatk-4.1.2.0/

In [ ]:
# retrieve the allele counts
# write a script that runs the ase read counter on each read and outputs a merged file for pandas/R

#may need to index and create sequence dict file
samtools faidx csi.chromosome.fa
samtools dict -o csi.chromosome.dict csi.chromosome.fa
java -jar picard.jar AddOrReplaceReadGroups \ I=CLV-102-1-GHA.bam \ O=rg.CLV-102-1-GHA.bam \ RGID=G21 \ RGLB=G21 \ RGPL=illumina \ RGPU=G21 \ RGSM=G21

#create a for loop for the rest of the files
id=(P21 G22 P22 G23 P23 P31 P32 P53 P81 P82 P83 PV1 PV2 PV3)

#remove the first line of fastq.list
# change it to a variable
sed '1d' fastq.list > input

list=($(cat input)) 


#create a for loop
# practice iterating the array
for i in $(seq 0 13); dojava -jar picard.jar AddOrReplaceReadGroups I=/data/transcriptome/${list[${i}]}.bam O:data/transcriptome/rg.${list[${i}]}.bam --RGID ${id[${i}]} --RGLB ${id[${i}]} --RGPL illumina --RGPU ${id[${i}]} --RGSM ${id[${i}]};done

#remove all indels from the file
sed -i 's/INDEL/' aln2-pe-snps-filt.vcf > snps.vcf

#index vcf file
gatk IndexFeatureFile -F /data/snps.vcf

#iterate the ase read for each file in bam
for file in $(cat input); do gatk ASEReadCounter -R csi.chromosome.fa -I rg.${file}.bam -V /data/snps.vcf -O ${file}.csv; done

#export results to python


In [1]:
import pandas as pd

# create dataframes and merge all of the file horizontally by chromosome and coordinate

In [2]:
# read all the different csv files as a dataframe
test = pd.read_csv('/RNA/VALENCIA/allele/CLV-102-1-GHA.csv', sep='\t', nrows=50, header=0, index_col=False )

# create a tuple to iterate from
list = ('CLV-102-1-GHA','CLV-102-1-PFA','CLV-102-2-GHA','CLV-102-2-PFA','CLV-102-3-GHA','CLV-102-3-PFA','CLV-103-1-PFA','CLV-103-2-PFA','CLV-105-3-PFA','CLV-108-1-PFA','CLV-108-2-PFA','CLV-108-3-PFA','PV5-1-PFA','PV5-2-PFA','PV5-3-PFA')

# dreate a list to enter each data frame into
df_list = []

# create a for loop to read each file as a dataframe
# use append() to enter values into an empty list
# f is needed to format list[i] into a string
for i in range(len(list)):
    df_list.append(pd.read_csv(f'/RNA/VALENCIA/allele/{list[i]}.csv', sep='\t', header=0, index_col=False))
    

In [3]:
# select only the columns that matter
merged = df_list[0][['contig','position','refAllele','altAllele','refCount','altCount']]


In [4]:
# merge dataframes based on a similar row
merged = merged.merge(df_list[1][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'), suffixes=('_1','_2'))

In [5]:
# repeat on the other dataframes
merged = merged.merge(df_list[2][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))

In [6]:
merged = merged.merge(df_list[3][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))

In [7]:
merged = merged.merge(df_list[4][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))

In [8]:
merged = merged.merge(df_list[5][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))
merged = merged.merge(df_list[6][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))
merged = merged.merge(df_list[7][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))
merged = merged.merge(df_list[8][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))
merged = merged.merge(df_list[9][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))
merged = merged.merge(df_list[10][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))
merged = merged.merge(df_list[11][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))
merged = merged.merge(df_list[12][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))
merged = merged.merge(df_list[13][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))
merged = merged.merge(df_list[14][['contig','position','refAllele','altAllele','refCount','altCount']], how='outer', on=('contig','position','refAllele','altAllele'))

In [10]:
# change the column names
merged.columns = ['contig','position','refAllele','altAllele','refCount_1','altCount_1','refCount_2','altCount_2','refCount_3','altCount_3','refCount_4','altCount_4','refCount_5','altCount_5','refCount_6','altCount_6','refCount_7','altCount_7','refCount_8','altCount_8','refCount_9','altCount_9','refCount_10','altCount_10','refCount_11','altCount_11','refCount_12','altCount_12','refCount_13','altCount_13','refCount_14','altCount_14','refCount_15','altCount_15']

#replace all NaN with 0s
merged = merged.fillna(0)
merged.to_csv('/RNA/VALENCIA/allele/merged.csv',sep='\t',index=False)
merged

,contig,position,refAllele,altAllele,refCount_1,altCount_1,refCount_2,altCount_2,refCount_3,altCount_3,...,refCount_11,altCount_11,refCount_12,altCount_12,refCount_13,altCount_13,refCount_14,altCount_14,refCount_15,altCount_15
0,chr1,367,G,C,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,chr1,368,G,T,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,chr1,9002,A,C,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,chr1,9011,G,A,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,chr1,9019,A,T,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,chr1,9027,C,T,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,chr1,9148,A,T,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,chr1,9162,C,A,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,chr1,9185,T,C,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,chr1,9209,C,T,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [189]:
del merged
del selected
del df_list
del test

# Visualize the allele read counts 

In [ ]:
# principal component analysis
